In [1]:
!pip3 install -q -U google-generativeai

In [1]:
import time
from tqdm import tqdm
import json
from dotenv import load_dotenv
import os
from requests.exceptions import HTTPError
load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [2]:
import google.generativeai as genai
import pandas as pd

genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel(model_name="gemini-1.5-flash")

/opt/miniconda3/envs/ml_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Set safety settings
safety_settings = [
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"}
]

In [3]:
df = pd.read_csv("csv_files/annotated_corrected_with_contexts.csv")
df.head()

In [149]:
df.columns = ['submissionId', 'submissionURL', 'submissionTitle', 'context1', 'context2', 'utterance','isTextSarcastic?', 'isImageSarcastic?', 'isTogetherSarcastic?']

In [150]:
df.head()

,submissionId,submissionURL,submissionTitle,context1,context2,utterance,isTextSarcastic?,isImageSarcastic?,isTogetherSarcastic?
0,1dncran,https://i.redd.it/1xvccvq6oi8d1.jpeg,the clouds did a thing this morning...,looks like someone laid out some rails of blow,all the deities got together are they are havi...,colombian clouds,1,0,1
1,c9tdn7,https://i.imgur.com/6KzNep5.jpg,looks like i won't be listening to my new viny...,it’s even crammed in there so hard your mailbo...,is there's a way to ask for a refund from the ...,how can an assigned delivery person not know t...,0,0,1
2,kctb6e,https://i.redd.it/9knprglwx3561.jpg,sounded way better in my head,i hope we get our 9-month cupcake in a cup.,bruh wall-e's my fav pixar movie! take my upvo...,i remember when it was just supposed to be a m...,0,1,1
3,oagyyo,https://i.redd.it/dm6sx4nqn9871.jpg,was just trying to help the driver.,orders $10 in food. tries to leave $5.50 tip. ...,more that?,maybe it prevents accidentally doing the wrong...,0,0,1
4,faw3bs,https://i.imgur.com/gwVDXvB.jpg,wow,"holy shit, i've seen a lot of these but that i...",the really boring dorian grey,step 1: dress up as subject.\n\nstep 2: walk u...,1,1,1


In [151]:
columns_to_keep = ['submissionId', 'submissionTitle', 'context1', 'context2', 'utterance','isTextSarcastic?']
df = df[columns_to_keep]

In [152]:
df.head()

,submissionId,submissionTitle,context1,context2,utterance,isTextSarcastic?
0,1dncran,the clouds did a thing this morning...,looks like someone laid out some rails of blow,all the deities got together are they are havi...,colombian clouds,1
1,c9tdn7,looks like i won't be listening to my new viny...,it’s even crammed in there so hard your mailbo...,is there's a way to ask for a refund from the ...,how can an assigned delivery person not know t...,0
2,kctb6e,sounded way better in my head,i hope we get our 9-month cupcake in a cup.,bruh wall-e's my fav pixar movie! take my upvo...,i remember when it was just supposed to be a m...,0
3,oagyyo,was just trying to help the driver.,orders $10 in food. tries to leave $5.50 tip. ...,more that?,maybe it prevents accidentally doing the wrong...,0
4,faw3bs,wow,"holy shit, i've seen a lot of these but that i...",the really boring dorian grey,step 1: dress up as subject.\n\nstep 2: walk u...,1


### Initializing new columns for llm generated annotations

In [153]:
df['ts_1'] = None
df['ts_2'] = None
df['ts_3'] = None

df.head()

,submissionId,submissionTitle,context1,context2,utterance,isTextSarcastic?,ts_1,ts_2,ts_3
0,1dncran,the clouds did a thing this morning...,looks like someone laid out some rails of blow,all the deities got together are they are havi...,colombian clouds,1,None,None,None
1,c9tdn7,looks like i won't be listening to my new viny...,it’s even crammed in there so hard your mailbo...,is there's a way to ask for a refund from the ...,how can an assigned delivery person not know t...,0,None,None,None
2,kctb6e,sounded way better in my head,i hope we get our 9-month cupcake in a cup.,bruh wall-e's my fav pixar movie! take my upvo...,i remember when it was just supposed to be a m...,0,None,None,None
3,oagyyo,was just trying to help the driver.,orders $10 in food. tries to leave $5.50 tip. ...,more that?,maybe it prevents accidentally doing the wrong...,0,None,None,None
4,faw3bs,wow,"holy shit, i've seen a lot of these but that i...",the really boring dorian grey,step 1: dress up as subject.\n\nstep 2: walk u...,1,None,None,None


In [163]:
def provide_prompt_for_text_labels(context, utterance):
    message = """
    TASK:
    You will be given a context and an utterance, and your task is to determine how sarcastic the given utterance is using the context.
    Rate each utterance on a scale of 0 to 10 for sarcasm, with 0 being completely non-sarcastic and 10 being extremely sarcastic.

    INSTRUCTIONS:
    The sacastic utterance must express the opposite of what is implied, often in a subtle manner.
    There should be a clear discrepancy between the literal statement and the intended meaning.
    Make sure the utterance implies a strong, unambiguous sarcastic impression, using the context provided. Otherwise, assign a very low sarcasm rating by default.
    Do not simply assume that the utterance is sarcastic just because it is humorous.
    
    Given the following few shot examples (delimited by <examples></examples>)
    <example>
    Context: so mr thompson, you say you have some programming skills???
    Utterance: probably referring to the "readability" certification. it's not hard, it just exempts you from needing a peer review on commits and enables you to approve other people's commits. totally overblown. i had readability in go, python, golang, and c when i worked there. you use an existing commit to request a readability certification.
    Your Response: {"Rating": "0.5"}
    ----

    Context: we’ve come full circle
    Utterance: there are three things you don’t discuss with strangers: religion, politics, and star wars.
    Your Response: {"Rating": "9.5"}
    ----

    Context: "clone wars? what are you talking about?"
    Utterance: i am one of the guys that saw star wars as a little boy (age 7, saw it 1978) the line was perfect!it immediately triggered my imagination of a dystopia where humans were cloned for being soldiers and maybe rebelled or tried to dominate which lead to a war. not far away from what was created later.
    Your Response: {"Rating": "0.7"}
    ----

    Context: keepers of peace, but in the most violent way
    Utterance: cut their hand off. that's the sith way. or go full obi and cut both their arms and legs off. that'll show em. bastards.
    Your Response: {"Rating": "8.7"}
    ----

    Context: people dont like videos cut too short.
    Utterance: then later, they would cut off just one wire from a 1,000,000 volt power grid
    Your Response: {"Rating": "2.3"}
    ----

    Context: roommates rice cooker has turned into brain-like tissue. he refuses to clean it, and leaves it on the shared kitchen counter.
    Utterance: that’s a biohazard.
    Your Response: {"Rating": "8.3"}
    ----

    Context: found a camera in my air b&b. it was halfway behind the painting with only the lens peeking out. discovered it because we heard it clicking after me and my girl got out of the shower.
    Utterance: that's not mildly infuriating, that's much worse.
    Your Response: {"Rating": "0.6"}
    ----

    Context: sick fuck fuses dogs togeter with black sorcery
    Utterance: i'm so confused rn
    Your Response: {"Rating": "8.4"}
    ----

    Context: this restaurant covered up the "no tip" option with a sticker to force tipping
    Utterance: percentage = 0%
    Your Response: {"Rating": "2.4"}
    ----

    Context: looks like i won't be listening to my new vinyl record. thanks, usps
    Utterance: how can an assigned delivery person not know the basic rule of don't bend the damn packages?!
    Your Response: {"Rating": "1.5"}
    </example>
    ----
    Context: someone put a paper trump sticker on my sister’s car because we have a biden sign in our front yard.
    Utterance: that’s so stupid. i don’t care if you’re a trump supporter or a biden supporter, do not vandalize other peoples property because they like a different candidate then you.
    Your Response: {"Rating": "0.6"}
    """

    message = message + f"""
    Generate a valid JSON for the following(delimited by <test></test>)
    <test>
    Context: {context}
    Utterance: {utterance}
    Your Response: 
    </test>
    Respond only with valid JSON. Do not write an introduction or summary.
    """
    
    return message

# Defining a response schema in order to obtain consistently formatted response
response_schema_text = {
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "Rating": {
                "type": "integer",
            },
        },
        "required": ["Rating"],
    },
}

def detect_sarcasm_in_text(context: str, utterance: str):
    response = model.generate_content(
        provide_prompt_for_text_labels(context, utterance),
        safety_settings=safety_settings,
        generation_config=genai.GenerationConfig(
            response_mime_type="application/json", 
            response_schema=response_schema_text, 
            temperature=0.8
        )
    )
    return response.text

In [164]:
detect_sarcasm_in_text("This is a nice view", "Take a pic")

# Label Generation

In [6]:
curIndex = 0

In [7]:
curIndex

0

In [ ]:
for i in tqdm(range(curIndex, df.shape[0])):
    time.sleep(4) # to deal with api limit error
    try:
        context = df.iloc[i]['submissionTitle']
        utterance = df.iloc[i]['utterance']
        
        # Attempt to detect sarcasm in the text
        response = detect_sarcasm_in_text(context, utterance)
        
        # Parse the response and update the DataFrame
        response_as_dict = json.loads(response)
        if len(response_as_dict) > 0:
            response_rating = response_as_dict[0].get('Rating')
            is_sarcastic = int(response_rating > 5)
            
    except Exception as e:
        print(e)
        is_sarcastic = 0
        
    finally:
        print("Response Rating: ", response_rating, " Original: ", df.iloc[i]['isTextSarcastic?'], "New: ", is_sarcastic)
        df.at[i, 'ts_1'] = is_sarcastic
        curIndex = i

In [172]:
df['isTextSarcastic?'].value_counts()

isTextSarcastic?
1    65
0    35
Name: count, dtype: int64

In [173]:
df['ts_1'].value_counts()

ts_1
1    90
0    10
Name: count, dtype: int64

# Evaluation

In [174]:
def compute_accuracy(y_true, y_pred):
    """
    Computes accuracy by comparing the true labels with the predicted labels.
    """
    correct_predictions = sum(y_t == y_p for y_t, y_p in zip(y_true, y_pred))
    accuracy = correct_predictions / len(y_true)
    return accuracy

def compute_precision(y_true, y_pred):
    """
    Computes precision by calculating the ratio of true positives to the sum of true and false positives.
    """
    true_positives = sum((y_t == 1 and y_p == 1) for y_t, y_p in zip(y_true, y_pred))
    predicted_positives = sum(y_p == 1 for y_p in y_pred)
    precision = true_positives / predicted_positives if predicted_positives > 0 else 0
    return precision

def compute_recall(y_true, y_pred):
    """
    Computes recall by calculating the ratio of true positives to the sum of true positives and false negatives.
    """
    true_positives = sum((y_t == 1 and y_p == 1) for y_t, y_p in zip(y_true, y_pred))
    actual_positives = sum(y_t == 1 for y_t in y_true)
    recall = true_positives / actual_positives if actual_positives > 0 else 0
    return recall

def compute_f1(precision, recall):
    """
    Computes F1 score as the harmonic mean of precision and recall.
    """
    if precision + recall == 0:
        return 0
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

def evaluate(y_true, y_pred):
    accuracy = compute_accuracy(y_true, y_pred)
    precision = compute_precision(y_true, y_pred)
    recall = compute_recall(y_true, y_pred)
    f1 = compute_f1(precision, recall)
    print("\tAccuracy: ", accuracy, "\n", "\tPrecision: ", precision,"\n", "\tRecall: ", recall, "\n", "\tF1 score: ", f1)
    

In [176]:
evaluate(df["isTextSarcastic?"].to_list(), df["ts_1"].to_list())

	Accuracy:  0.71 
 	Precision:  0.7 
 	Recall:  0.9692307692307692 
 	F1 score:  0.8129032258064517
